In [1]:
# %%writefile crawler.py

# !pip3 install -q selenium bs4
# !pip3 install jdatetime
# !pip3 install khayyam
!pip3 install -q urllib3
# !pip install jupyter_contrib_nbextensions
# !pip install nbresuse
# !pip install selenium, bs4
# from selenium import webdriver
# from bs4 import BeautifulSoup as bs
from requests import Session, Request
import pickle
import time
from pathlib import Path
from datetime import date, timedelta
from khayyam import JalaliDate, JalaliDatetime
from threading import Thread
from queue import Queue
import multiprocessing


parentPageUrl = 'http://www.tsetmc.com/Loader.aspx?ParTree=15131F'
saveLocation = "../xcels/"
xcelBaseUrl = "http://members.tsetmc.com/tsev2/excel/MarketWatchPlus.aspx?d="
session = Session()
request = Request("Get", parentPageUrl)
prepared = session.prepare_request(request)
respond = session.send(prepared, verify=False)


def fetch(saveLocation, year, month, day):
    global xcelBaseUrl
    
    with open(saveLocation + str(year) + "-" + str(month) + "-" + str(day) + ".xlsx", 'wb') as f:
        xcelUrl = xcelBaseUrl + str(year) + "/" + str(month) + "/" + str(day)
        xcelRequest = Request("Get", xcelUrl)
        xcelPrepared = session.prepare_request(xcelRequest)
        xcelRespond = session.send(xcelPrepared, verify=False, stream=True)
        xcelRespond.raise_for_status()
        for chunk in xcelRespond.iter_content(chunk_size=8192): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
            # f.flush()

def crawl(startDate, endDate):
    global saveLocation
    
    now = startDate
    while now <= endDate:
        jalaldate = JalaliDate(now)
        year = jalaldate.year
        month = jalaldate.month
        day = jalaldate.day
        if jalaldate.weekday() < 5 and not Path(saveLocation + str(year) + "-" + str(month) + 
                                                "-" + str(day) + ".xlsx").is_file():
            fetch(saveLocation=saveLocation, year=year, month=month, day=day)
            time.sleep(5)
        now = now + timedelta(days=1)
        
def crawlingThread(threadName, startDate):
    endDateJalali = JalaliDate(year=1398, month=6, day=5)
    while True:
        if endDateJalali != JalaliDate.today():
            endDateJalali = JalaliDate.today()
            crawl(startDate, endDateJalali.todate())
            startDate = endDateJalali.todate()
            with open(saveLocation + "crawlstat", 'w') as statfile:
                lastcheck = JalaliDatetime.now()
                statfile.write("last check")
                statfile.write(str(lastcheck))
                statfile.write("\n")
                statfile.write("last crawl: ")
                statfile.write(str(endDateJalali))
                statfile.write("\n")
        time.sleep(24 * 3600)

# startDate = JalaliDate(year=1380, month=1, day=1).todate()
startDate = JalaliDate(year=1398, month=7, day=10).todate()
crawlThread = multiprocessing.Process(target=crawlingThread, args=("Thread-crawl", startDate))
crawlThread.start()

In [6]:
# file_object = open("user.pkl", 'ab')
# # pickle.dump(requests.utils.dict_from_cookiejar(resp.cookies), file_object)

# soup = bs(resp.text, 'html.parser')
# name = soup.find_all(itemprop='name')
# name = [x.get_text() for x in name]
# moredis = soup.findAll("p", {"class": "prd-desc more"})
# moredis = [x.get_text() for x in moredis]
# weight = soup.findAll("span", {"class": "unit"})
# weight = [x.get_text() for x in weight]

# import datetime
# now = datetime.datetime.now()

# with open('somefile') as f:
#     cookies = requests.utils.cookiejar_from_dict(pickle.load(f))
#     session = requests.session(cookies=cookies)

# from http.cookies import SimpleCookie
# rawdata = 'Cookie: _ga=GA1.2.969355752.1534482039;' \
#           ' JSESSIONID=nM0Rmitqyzs4sWMMdOBXB5BDbq09pS3ilLqwUJOV1bCZv_BxQgHS!484861232'
# cookie = SimpleCookie()
# cookie.load(rawdata)
